<a href="https://colab.research.google.com/github/srujith7/AD-075/blob/main/7_FEB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
Text="I love NLP and I will learn NLP within 2 months."

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer=CountVectorizer()
vectorizer.fit([Text])
vectorizer=vectorizer.transform([Text])
#print(vectorizer.vocabulary_)
print(vectorizer.toarray())

[[1 1 1 1 2 1 1]]


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Text variable containing the document
Text = "I love NLP and I will learn NLP within 2 months."

# Create a CountVectorizer instance
vectorizer = CountVectorizer()

# Fit the vectorizer to a list containing the document
# The fit method expects an iterable, so we enclose the Text variable in a list
vectorizer.fit([Text])

# Transform the document into a vector representation
vector = vectorizer.transform([Text]) # The transform method also expects an iterable

In [ ]:
print(vectorizer.vocabulary_)
print(vector.toarray())

{'love': 2, 'nlp': 4, 'and': 0, 'will': 5, 'learn': 1, 'within': 6, 'months': 3}
[[1 1 1 1 2 1 1]]


In [ ]:
print(vector)

  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 3)	1
  (0, 4)	2
  (0, 5)	1
  (0, 6)	1


In [ ]:
import pandas as pd # Import the pandas library and assign it to the alias 'pd'

df = pd.DataFrame(vector.toarray(), columns=vectorizer.get_feature_names_out()) # Create the dataframe
df # Display the dataframe

,and,learn,love,months,nlp,will,within
0,1,1,1,1,2,1,1


In [ ]:
text = 'I am learning NLP'

In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from textblob import TextBlob
TextBlob(text).ngrams(1)  # Use TextBlob with a capital 'B'

[WordList(['I']), WordList(['am']), WordList(['learning']), WordList(['NLP'])]

In [ ]:
TextBlob(text).ngrams(5)

[]

In [1]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("/content/covid_fake.csv")

In [3]:
df.shape

(1164, 4)

In [4]:
df['label'].value_counts()

,count
label,
TRUE,584
Fake,345
fake,230


In [5]:
df.loc[df['label']=='FAKE',['label']]=='FAKE'
df.loc[df['label']=='fake',['label']]=='FAKE'
df.loc[df['source']=='facebook',['label']]=='FACEBOOK'
df.text.fillna(df.title,inplace=True)
df.loc[5]['label']='FAKE'
df.loc[15]['label']='TRUE'
df.loc[43]['label']='FAKE'
df.loc[131]['label']='TRUE'
df.loc[242]['label']='FAKE'

In [13]:
df.shape
df.loc[5:15]

,title,text,source,label,title_text
5,CORONA UNMASKED: Chinese Intelligence Officer ...,CORONA UNMASKED: Chinese Intelligence Officer ...,NaN,FAKE,corona unmasked chinese intelligence officer r...
6,NaN,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,Fake,urgent health bulletin to the public ministry...
7,NaN,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,Fake,pls tell ur families relatives and friendsmoh...
8,NaN,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,Fake,serious excellent advice by japanese doctors ...
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,TRUE,basic protective measures against the new coro...
10,NaN,The new Coronavirus may not show signs of infe...,Taiwan Experts,Fake,the new coronavirus may not show signs of inf...
11,NaN,A vaccine meant for cattle can be used to figh...,facebook,Fake,a vaccine meant for cattle can be used to fig...
12,NaN,Using a hair dryer to breathe in hot air can c...,Youtube,Fake,using a hair dryer to breathe in hot air can ...
13,NaN,Corona virus before it reaches the lungs it re...,twitter,Fake,corona virus before it reaches the lungs it r...
14,Exposing yourself to the sun or to temperature...,"You can catch COVID-19, no matter how sunny or...",https://www.who.int/emergencies/diseases/novel...,TRUE,exposing yourself to the sun or to temperature...


In [12]:
def preprocessor(text):
  text=re.sub('<[^>]*>','',text)
  # Removing the problematic re.findall line
  # Instead of findall, we could use re.sub to remove the characters
  text = re.sub(r'[^\w\s]', '', text)  # Replace unwanted characters with empty string
  text=re.sub(r'[\n]','',text)
  text=text.lower()
  return text

df['title_text']=df['title_text'].apply(preprocessor)
df['title_text'][3]

' the corona virus is a man made virus created in a wuhan laboratory ask billgates who financed it'

In [16]:
porter = PorterStemmer()
def tokenizer(text):
  return text.split()

def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

In [11]:
tfidf = TfidfVectorizer(strip_accents=None,lowercase=False,preprocessor=None)
y = df.label.values
X = tfidf.fit_transform(df.text)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.5, shuffle=False)

In [15]:
clf = LogisticRegressionCV(cv=5,scoring='accuracy',random_state=0,n_jobs=-1,verbose=3,max_iter=300).fit(x_train,y_train)
fake_news_model = open("fake_news_model.pkl","wb")
pickle.dump(clf,fake_news_model)
fake_news_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.1min finished


In [18]:
from sklearn.metrics import accuracy_score, classification_report
prediction = clf.predict(x_test)
y_pred = clf.predict(x_test)
print(classification_report(y_test,y_pred))
accuracy_score(y_test,prediction)

              precision    recall  f1-score   support

        Fake       0.58      0.71      0.64       139
        TRUE       0.95      0.90      0.92       390
        fake       0.25      0.21      0.23        53

    accuracy                           0.79       582
   macro avg       0.59      0.60      0.60       582
weighted avg       0.80      0.79      0.79       582



0.7903780068728522